# 🧬 Demo: Genetic Algorithm Individual Class

This notebook demonstrates the functionality of the `Individual` abstract base class and a concrete subclass `BinaryIndividual`. The goal is to show how this flexible design supports mutation, crossover, fitness evaluation, and operator overloading for use in genetic algorithms.

## 🧱 Class Overview

- `Individual`: An abstract class that defines a genetic algorithm individual.
  - Includes pluggable methods: `random_representation`, `mutation`, `crossover`, `calculate_fitness`, and `semantic_key`.
  - Overloads key operators for convenience:
    - `@` → Crossover (e.g., `child = parent1 @ parent2`)
    - `**` → Multiple self-crossovers (e.g., `clones = parent ** 3`)
    - `()` → Evaluate fitness (e.g., `fitness = individual()`)
    - Comparison (`<`, `>`, etc.) based on fitness
    - Hashing and equality via `semantic_key` for use in sets and dictionaries

- `BinaryIndividual`: A concrete implementation where the genome is a list of 0s and 1s.
  - Fitness is defined as the number of 1s (maximize ones).
  - Mutation flips a single bit.
  - Crossover combines parts of the genome from two parents.

## ✅ Features Demonstrated

1. **Random Initialization**
   - Create individuals with random binary genomes.

2. **Fitness Evaluation**
   - Automatically lazily computed and cached.

3. **Mutation**
   - Use `mutation()` to modify an individual in-place.

4. **Crossover**
   - Use the `@` operator (or `crossover()`) to generate a child from two parents.

5. **Iteration**
   - Use a `for` loop to generate several mutated versions from an individual.

6. **Operator Overloading**
   - Use Python syntax sugar for fitness, crossover, and comparisons.

7. **Sorting Individuals**
   - Individuals are comparable by fitness, so they can be sorted.

8. **Using Sets**
   - `Individual` instances are hashable via `semantic_key`, allowing deduplication in sets or as dictionary keys.

This modular and extensible framework allows for easy implementation of different genome types and evolutionary strategies.

In [2]:
from abc import ABC, abstractmethod
from copy import deepcopy
import random

In [ ]:
# import a script in another directory
import sys
import os
from typing import List, Callable
import pandas as pd
relative_path = os.path.join("..", "Classes")
print(sys.path)
sys.path.append(relative_path)
#sorted(os.listdir(relative_pathhgggggfffddwwddssddbbbbbbeeeeeeeaaaaaaaaaatttttttrrrrrrriiiiiizzzzzzzzzzzzz))
from Individual import Individual

['/opt/anaconda3/envs/Cifo_RunningDinner/lib/python312.zip', '/opt/anaconda3/envs/Cifo_RunningDinner/lib/python3.12', '/opt/anaconda3/envs/Cifo_RunningDinner/lib/python3.12/lib-dynload', '', '/opt/anaconda3/envs/Cifo_RunningDinner/lib/python3.12/site-packages']


In [ ]:
from ga_runner import genetic_algorithm_runs_to_csv, tournament_selection
from BinaryIndividual import BinaryIndividual      # your concrete class

# ❶  Factory that builds a fresh population for *every* run
POP_SIZE = 50
pop_factory = lambda: [BinaryIndividual() for _ in range(POP_SIZE)]

# ❷  Wrap the selector so its signature matches genetic_algorithm(pop, maximization)
selector = lambda pop, maxim: tournament_selection(pop, k=5, minimise=not maxim)

# ❸  Launch, say, 3 runs of 100 generations each
genetic_algorithm_runs_to_csv(
    run_count=3,
    pop_factory=pop_factory,
    max_gen=100,
    selection_algorithm=selector,
    csv_path="ga_runs.csv",
    maximization=False,     # set True if higher fitness is better
    verbose=True
)

# -> 'ga_runs.csv' now contains:
#    Gen_1,Best_1,Gen_2,Best_2,Gen_3,Best_3
#     1,0.883,1,0.901,1,0.879
#     2,0.872,2,0.893,2,0.866
#     ...


In [4]:
# selection.py  
def tournament_selection(
    population: List[Individual],
    k: int = 3,
    minimise: bool = True
) -> Individual:
    """
    k-way tournament (with replacement). Returns the best contestant.

    Parameters
    ----------
    population : List[Individual]
        Current population (e.g., list of BinaryIndividual objects).
    k : int, default 3
        Tournament size.
    minimise : bool, default True
        Set False if higher fitness is better.

    Returns
    -------
    Individual
        The tournament winner.
    """
    contestants = random.choices(population, k=k)
    key_fn = (lambda ind: ind.fitness) if minimise else (lambda ind: -ind.fitness)
    return min(contestants, key=key_fn)


def rank_selection(
    population: List[Individual],
    minimise: bool = True
) -> Individual:
    """
    Rank-based roulette selection.

    1. Sort population by fitness.
    2. Assign weights N, N-1, …, 1 (best gets highest weight).
    3. Sample one individual proportionally to its weight.

    Parameters
    ----------
    population : List[Individual]
        Current population.
    minimise : bool, default True
        Set False if higher fitness is better.

    Returns
    -------
    Individual
        The selected individual.
    """
    ranked = sorted(population,
                    key=lambda ind: ind.fitness,
                    reverse=not minimise)   # best first
    n = len(ranked)
    weights = list(range(n, 0, -1))
    return random.choices(ranked, weights=weights, k=1)[0]


In [5]:
class BinaryIndividual(Individual):
    def __init__(self, genome_length=10):
        super().__init__()
        self.genome_length = genome_length
        self.genome = [0] * genome_length
        self.random_representation()

    def random_representation(self):
        self.genome = [random.choice([0, 1]) for _ in range(self.genome_length)]

    def check_representation(self):
        return all(g in [0, 1] for g in self.genome)

    def calculate_fitness(self):
        return sum(self.genome)  # Maximize number of 1s

    def mutation(self):
        idx = random.randint(0, self.genome_length - 1)
        self.genome[idx] = 1 - self.genome[idx]  # Flip bit
        return self

    def crossover(self, other):
        pivot = random.randint(1, self.genome_length - 1)
        child = self.copy_Individual()
        child.genome = self.genome[:pivot] + other.genome[pivot:]
        return child

    def semantic_key(self):
        return tuple(self.genome)

# Testing Features

## Create two individuals


In [6]:
ind1 = BinaryIndividual()
ind2 = BinaryIndividual()

print("Individual 1:", ind1.genome)
print("Fitness 1:", float(ind1))

print("Individual 2:", ind2.genome)
print("Fitness 2:", float(ind2))

Individual 1: [1, 0, 1, 1, 1, 1, 0, 1, 0, 0]
Fitness 1: 6.0
Individual 2: [1, 0, 1, 0, 0, 0, 0, 1, 0, 1]
Fitness 2: 4.0


## Crossover


In [7]:
child = ind1 @ ind2
print("\nChild (crossover):", child.genome)
print("Fitness (child):", float(child))


Child (crossover): [1, 0, 1, 1, 1, 1, 0, 1, 0, 1]
Fitness (child): 7.0


## Mutation


In [8]:
mutant = ind1.copy_Individual()
mutant.mutation()
print("\nMutated Individual:", mutant.genome)
print("Fitness (mutated):", float(mutant))


Mutated Individual: [1, 0, 1, 0, 1, 1, 0, 1, 0, 0]
Fitness (mutated): 5.0


## Iteration (generate mutated versions)

In [9]:
print("\nMultiple Mutations:")
for i, m in enumerate(ind1):
    print(f"Mutation {i+1}:", m.genome, "Fitness:", float(m))


Multiple Mutations:
Mutation 1: [1, 0, 1, 1, 1, 1, 0, 1, 0, 1] Fitness: 7.0
Mutation 2: [1, 0, 1, 1, 1, 1, 1, 1, 0, 0] Fitness: 7.0
Mutation 3: [1, 0, 1, 1, 1, 1, 0, 0, 0, 0] Fitness: 5.0
Mutation 4: [1, 0, 1, 1, 1, 1, 0, 1, 1, 0] Fitness: 7.0
Mutation 5: [1, 1, 1, 1, 1, 1, 0, 1, 0, 0] Fitness: 7.0


## Operator Overloading

In [10]:
print("\nUsing ** Operator for Self-Crossover")
clones = ind1 ** 3
for i, c in enumerate(clones):
    print(f"Clone {i+1}:", c.genome)


Using ** Operator for Self-Crossover
Clone 1: [1, 0, 1, 1, 1, 1, 0, 1, 0, 0]
Clone 2: [1, 0, 1, 1, 1, 1, 0, 1, 0, 0]
Clone 3: [1, 0, 1, 1, 1, 1, 0, 1, 0, 0]


## Comparison


In [11]:
print("\nIs Individual 1 better than Individual 2?", ind1 > ind2)


Is Individual 1 better than Individual 2? True


## Sorting and Deduplication via Set

In [12]:
print("\n=== Sorting Individuals by Fitness ===")
individuals = [BinaryIndividual() for _ in range(5)]

for i, ind in enumerate(individuals):
    print(f"Ind {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

sorted_individuals = sorted(individuals)
print("\nSorted by fitness (ascending):")
for i, ind in enumerate(sorted_individuals):
    print(f"Rank {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")


=== Sorting Individuals by Fitness ===
Ind 1: Genome = [0, 1, 1, 0, 1, 0, 1, 0, 1, 0], Fitness = 5.0
Ind 2: Genome = [1, 1, 0, 1, 1, 0, 0, 1, 1, 1], Fitness = 7.0
Ind 3: Genome = [0, 0, 0, 0, 1, 0, 0, 0, 0, 1], Fitness = 2.0
Ind 4: Genome = [1, 0, 1, 1, 1, 1, 0, 0, 1, 0], Fitness = 6.0
Ind 5: Genome = [1, 1, 1, 0, 1, 0, 0, 0, 0, 0], Fitness = 4.0

Sorted by fitness (ascending):
Rank 1: Genome = [0, 0, 0, 0, 1, 0, 0, 0, 0, 1], Fitness = 2.0
Rank 2: Genome = [1, 1, 1, 0, 1, 0, 0, 0, 0, 0], Fitness = 4.0
Rank 3: Genome = [0, 1, 1, 0, 1, 0, 1, 0, 1, 0], Fitness = 5.0
Rank 4: Genome = [1, 0, 1, 1, 1, 1, 0, 0, 1, 0], Fitness = 6.0
Rank 5: Genome = [1, 1, 0, 1, 1, 0, 0, 1, 1, 1], Fitness = 7.0


## Storing Unique Individuals in a Set

In [13]:
print("\n=== Deduplication with Set ===")
ind1 = BinaryIndividual()
ind2 = ind1.copy_Individual(delete_fitness=False)  # Exact copy

# Mutate a third individual to likely get a different genome
ind3 = BinaryIndividual()
ind3.mutation()

ind_set = {ind1, ind2, ind3}

for i, ind in enumerate(ind_set):
    print(f"Unique {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

print(f"\nTotal unique individuals: {len(ind_set)}")


=== Deduplication with Set ===
Unique 1: Genome = [1, 1, 0, 0, 0, 1, 1, 1, 1, 0], Fitness = 6.0
Unique 2: Genome = [0, 1, 0, 0, 1, 1, 0, 0, 1, 0], Fitness = 4.0

Total unique individuals: 2


## Selection Mechanisms

In [14]:
# Parameter values only for illustration
POP_SIZE = 50

population = [BinaryIndividual() for _ in range(POP_SIZE)]

In [46]:
# run the two selection mechanisms
winner_tournament = tournament_selection(population, k=5)   # play with k
winner_rank       = rank_selection(population)

print("Tournament winner fitness :", winner_tournament.fitness)
print("Rank-based winner fitness :", winner_rank.fitness)

Tournament winner fitness : 3
Rank-based winner fitness : 4


In [57]:
import numpy as np

def sample(selector, n=50):
    return np.mean([selector(population).fitness for _ in range(n)])

print("Avg fitness (tournament):", sample(lambda pop: tournament_selection(pop, k=3)))
print("Avg fitness (rank)      :", sample(rank_selection))


Avg fitness (tournament): 3.64
Avg fitness (rank)      : 4.18


## Run GA

In [ ]:
from Solution import Solution

def genetic_algorithm(
    initial_population: list[Solution],
    max_gen: int,
    selection_algorithm: Callable,
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.2,
    elitism: bool = True,
    verbose: bool = False,
):
    """
    Executes a genetic algorithm to optimize a population of solutions.

    Args:
        initial_population (list[Solution]): The starting population of solutions.
        max_gen (int): The maximum number of generations to evolve.
        selection_algorithm (Callable): Function used for selecting individuals.
        maximization (bool, optional): If True, maximizes the fitness function; otherwise, minimizes. Defaults to False.
        xo_prob (float, optional): Probability of applying crossover. Defaults to 0.9.
        mut_prob (float, optional): Probability of applying mutation. Defaults to 0.2.
        elitism (bool, optional): If True, carries the best individual to the next generation. Defaults to True.
        verbose (bool, optional): If True, prints detailed logs for debugging. Defaults to False.

    Returns:
        Solution: The best solution found on the last population after evolving for max_gen generations.
        list[float]: The fitness of the best individual over the generations
    """
    best_fitness_over_gens = []

    # 1. Initialize a population with N individuals
    population = initial_population

    # 2. Repeat until termination condition
    for gen in range(1, max_gen + 1):
        if verbose:
            print(f'-------------- Generation: {gen} --------------')

        # 2.1. Create an empty population P'
        new_population = []

        # 2.2. If using elitism, insert best individual from P into P'
        if elitism:
            new_population.append(deepcopy(get_best_ind(population, maximization)))
        
        # 2.3. Repeat until P' contains N individuals
        while len(new_population) < len(population):
            # 2.3.1. Choose 2 individuals from P using a selection algorithm
            first_ind = selection_algorithm(population, maximization)
            second_ind = selection_algorithm(population, maximization)

            if verbose:
                print(f'Selected individuals:\n{first_ind}\n{second_ind}')

            # 2.3.2. Choose an operator between crossover and replication
            # 2.3.3. Apply the operator to generate the offspring
            if random.random() < xo_prob:
                offspring1, offspring2 = first_ind.crossover(second_ind)
                if verbose:
                    print(f'Applied crossover')
            else:
                offspring1, offspring2 = deepcopy(first_ind), deepcopy(second_ind)
                if verbose:
                    print(f'Applied replication')
            
            if verbose:
                print(f'Offspring:\n{offspring1}\n{offspring2}')
            
            # 2.3.4. Apply mutation to the offspring
            first_new_ind = offspring1.mutation(mut_prob)
            # 2.3.5. Insert the mutated individuals into P'
            new_population.append(first_new_ind)

            if verbose:
                print(f'First mutated individual: {first_new_ind}')
            
            if len(new_population) < len(population):
                second_new_ind = offspring2.mutation(mut_prob)
                new_population.append(second_new_ind)
                if verbose:
                    print(f'Second mutated individual: {first_new_ind}')
        
        # 2.4. Replace P with P'
        population = new_population

        if verbose:
            print(f'Final best individual in generation: {get_best_ind(population, maximization).fitness()}')
        
        best_ind = get_best_ind(population, maximization)
        best_fitness_over_gens.append(best_ind.fitness())

    # 3. Return the best individual in P + the best individual fitness over generations
    return get_best_ind(population, maximization), best_fitness_over_gens

In [ ]:
# run GA many times & collect results
def genetic_algorithm_runs_to_csv(
    run_count: int,
    pop_factory: Callable[[], List[Individual]],
    max_gen: int,
    selection_algorithm: Callable,
    csv_path: str = "ga_results.csv",
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.2,
    elitism: bool = True,
    verbose: bool = False,
):
    """
    Executes the GA `run_count` times.  After *each* run it appends two
    columns to `csv_path`:

        Gen_<run_id>   – generation numbers 1 … max_gen
        Best_<run_id>  – best fitness in that generation

    Existing columns are never overwritten; new runs get the next free id.
    The function returns the final DataFrame for convenience.
    """

    # load the results table
    if os.path.isfile(csv_path):
        df = pd.read_csv(csv_path)
        # count how many runs are already stored
        existing = len([c for c in df.columns if c.startswith("Best_")])
    else:
        # first time: start with an empty table
        df = pd.DataFrame()
        existing = 0

    # guaranteed generation index for *all* runs
    gen_index = list(range(1, max_gen + 1))

    # Execute the runs
    for run_offset in range(run_count):
        run_id = existing + run_offset + 1          # 1-based, monotonic

        # Create fresh initial population
        population = pop_factory()

        # Evolve it
        _, best_trace = genetic_algorithm(
            initial_population=population,
            max_gen=max_gen,
            selection_algorithm=selection_algorithm,
            maximization=maximization,
            xo_prob=xo_prob,
            mut_prob=mut_prob,
            elitism=elitism,
            verbose=verbose,
        )

        # Append two new columns
        df[f"Gen_{run_id}"]  = gen_index
        df[f"Best_{run_id}"] = best_trace

        # (optional) autosave after every run
        df.to_csv(csv_path, index=False)

        if verbose:
            print(f"Run {run_id} finished – results written to '{csv_path}'")

    return df